In [20]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import datetime

In [21]:
df = pd.read_csv("Predict_ready_df")

In [22]:
from sklearn.preprocessing import FunctionTransformer

def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))


def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

hour_arr = np.array(df['hour'])
day_arr = np.array(df['day_of_week'])

df['hour_sin'] = sin_transformer(24).fit_transform(hour_arr.reshape(-1,1))
df['hour_cos'] = cos_transformer(24).fit_transform(hour_arr.reshape(-1,1))
df['day_of_week_sin'] = sin_transformer(24).fit_transform(day_arr.reshape(-1,1))
df['day_of_week_cos'] = cos_transformer(24).fit_transform(day_arr.reshape(-1,1))

In [23]:
df

,Unnamed: 0,timestamp,station_id,bike_id,idle_time,temperature,rain,hour,day_of_week,lat,lon,close_to_some_bus_station,close_to_some_schools,close_to_some_leisure_spots,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos
0,0,2019-01-01 01:00:00,3,5426,0.000000,12.0,0,1,1,37.78638,-122.40490,0,0,1,2.588190e-01,0.965926,0.258819,0.965926
1,1,2019-01-01 09:00:00,3,2191,113.283333,10.0,0,9,1,37.78638,-122.40490,0,0,1,7.071068e-01,-0.707107,0.258819,0.965926
2,2,2019-01-01 09:00:00,3,5063,107.800000,10.0,0,9,1,37.78638,-122.40490,0,0,1,7.071068e-01,-0.707107,0.258819,0.965926
3,3,2019-01-01 09:00:00,3,5262,98.400000,10.0,0,9,1,37.78638,-122.40490,0,0,1,7.071068e-01,-0.707107,0.258819,0.965926
4,4,2019-01-01 10:00:00,3,5354,31.116667,9.0,0,10,1,37.78638,-122.40490,0,0,1,5.000000e-01,-0.866025,0.258819,0.965926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3132766,3132766,2019-12-23 17:00:00,498,353685,975.483333,10.0,0,17,0,37.72589,-122.45244,0,1,1,-9.659258e-01,-0.258819,0.000000,1.000000
3132767,3132767,2019-12-24 10:00:00,498,10310,196.016667,8.0,0,10,1,37.72589,-122.45244,0,1,1,5.000000e-01,-0.866025,0.258819,0.965926
3132768,3132768,2019-12-30 14:00:00,498,10962,0.383333,9.0,0,14,0,37.72589,-122.45244,0,1,1,-5.000000e-01,-0.866025,0.000000,1.000000
3132769,3132769,2019-12-30 14:00:00,498,10962,1252.866667,9.0,0,14,0,37.72589,-122.45244,0,1,1,-5.000000e-01,-0.866025,0.000000,1.000000


In [24]:
xp = df[['temperature','rain','hour_sin','hour_cos','close_to_some_bus_station','close_to_some_schools','close_to_some_leisure_spots','hour',
         'day_of_week_sin','day_of_week_cos']].values
yp= df['idle_time'].values

In [25]:
X_train_val, X_test, y_train_val, y_test = train_test_split(xp, yp, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42)

Scaling features

In [26]:

from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
X_train[:, 0] = scaler.fit_transform(X_train[:, 0].reshape(-1, 1)).flatten()
X_test[:, 0] = scaler.transform(X_test[:, 0].reshape(-1, 1)).flatten()



In [29]:
from sklearn.preprocessing import PolynomialFeatures
degree = 3
poly = PolynomialFeatures(degree= degree,include_bias= False)
poly_test_x = poly.fit_transform(X_test)

In [30]:
poly_test_x

array([[ 1.04412224,  0.        , -0.70710678, ...,  0.24148146,
         0.06470476,  0.01733759],
       [ 0.57043601,  0.        , -0.8660254 , ...,  0.06470476,
         0.24148146,  0.90122107],
       [-0.61377954,  1.        ,  0.5       , ...,  0.21650635,
         0.375     ,  0.64951905],
       ...,
       [ 2.70202401,  0.        , -0.8660254 , ...,  0.21650635,
         0.375     ,  0.64951905],
       [ 1.75465157,  0.        , -0.96592583, ...,  0.        ,
         0.        ,  1.        ],
       [-1.08746576,  0.        ,  0.5       , ...,  0.        ,
         0.        ,  1.        ]])

In [18]:
from sklearn.linear_model import Lasso

model_L1 = Lasso(alpha = 0.01)

model_L1.fit(poly_test_x, y_test)
predict = model_L1.predict(poly_test_x)

print("Coefficients ", model_L1.coef_, "\nIntercept ", model_L1.intercept_ )

c:\Users\riven\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.751e+10, tolerance: 8.518e+06
  model = cd_fast.enet_coordinate_descent(


Coefficients  [-1.29556252e+01 -0.00000000e+00 -4.07505838e+01  1.52550564e+02
  4.83251155e+02 -0.00000000e+00 -1.31440041e+01 -8.09273837e+00
  9.86452526e+01 -2.15086948e+00  1.43147069e+00  3.04571793e+00
  4.45166188e+00 -7.56373996e+00 -2.19573034e+00  1.29457942e+00
 -6.88415413e+00  5.41627213e-01  7.52219586e+00  5.39910111e+00
 -0.00000000e+00  7.45243706e+00 -2.59343111e+00 -2.88228568e+01
 -1.22220020e+01  2.95483389e+01  5.78267711e-01 -1.89417671e+01
  1.19884433e+01 -9.93077488e+01  5.28980911e+00  1.03713081e+01
  1.96409179e+01  1.29086571e+01  9.14056919e-01 -2.84509615e+01
 -4.04140144e+00  0.00000000e+00 -2.07705900e+01  3.04482224e+01
 -4.80676209e+00  4.35468058e+00 -5.52047960e+01 -9.61885780e+01
  0.00000000e+00  9.55583105e+01 -5.26239306e+02  1.76056828e-01
  7.78140545e+00 -3.97312836e+00 -5.00220703e+00 -9.91627857e-15
  2.51074130e-01  4.11272613e+01  8.45407847e+01 -0.00000000e+00
  4.90267577e-01 -2.00026173e+01 -4.29841200e+01  3.95105075e-01
 -7.1726455